In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv("dataset_full_big3.csv")

In [3]:
df['alzheimer'].value_counts()

0    677
1     52
2     41
Name: alzheimer, dtype: int64

In [4]:
from IPython.display import display
with pd.option_context('display.max_rows', 5, 'display.max_columns', 38):
    display(df)

,Unnamed: 0,code,alzheimer,age,education_x,ws_big_big3_pauses,ws_big_big3_pauses_long,ws_big_big3_pauses_short,ws_big_big3_uniquewords,ws_big_big3_uniquewordsnorm,ws_big_big3_wordcount,big3_asr_correct_word_time,big3_asr_reaction_time,big3_asr_speech_rate_syllables_per_second,big3_asr_speech_rate_words_per_second,big3_asr_syllable_count,big3_asr_tags_[@:],big3_asr_tags_[@],big3_asr_tags_[fil],big3_asr_tags_[int],big3_asr_tags_[spk],big3_asr_utterance_count,big3_asr_wav_duration,big3_asr_word_count,big3_asr_word_count_unique,diagnosis,moca,education_y,big3,big3_2,big3_pauses,big3_tags,big3_lema,big3_2_tags,big3_pauses_tags,big3_first_tag,big3_2_first_tag,big3_pauses_first_tag
0,0,3fiigiv401,0,52,2,25,17,8,81,81,130,-1,1.140197,4.298944,2.319807,265,0,2,4,0,2,1,90.015625,143,94,1.0,26.0,15.0,a muž v kuchyni varí a má aj pomocníka ten doč...,a muž v kuchyni varí a má aj pomocníka [spk] [...,[LONGSIL] a muž v kuchyni varí a má aj pomocní...,O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T SSms4 PFm...,a|s muž v kuchyňa variť a|s mať aj pomocník t...,O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T SSms4 xsp...,xlongsilx O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T...,OOOO SSSS EEEE SSSS VVVV OOOO VVVV TTTT SSSS ...,OOOO SSSS EEEE SSSS VVVV OOOO VVVV TTTT SSSS ...,xlongsilx OOOO SSSS EEEE SSSS VVVV OOOO VVVV ...
1,1,ysw9a5m701,0,66,2,23,9,14,97,97,151,-1,1.531078,4.236261,2.443997,286,0,2,12,0,0,1,90.026313,165,99,1.0,28.0,13.0,a ja aj toto je toto je by mohol nevie dal ale...,a ja aj toto je toto je by mohol nevie dal ale...,[LONGSIL] a ja aj toto je toto je by [SHORTSIL...,O PPhs1 T PFns1 VKesc+ PFns1 VKesc+ Y VLesci+...,a|s ja aj toto byť|jesť toto byť|jesť by môcť...,O PPhs1 T PFns1 VKesc+ PFns1 VKesc+ Y VLesci+...,xlongsilx O PPhs1 T PFns1 VKesc+ PFns1 VKesc+...,OOOO PPPP TTTT PPPP VVVV PPPP VVVV YYYY VVVV ...,OOOO PPPP TTTT PPPP VVVV PPPP VVVV YYYY VVVV ...,xlongsilx OOOO PPPP TTTT PPPP VVVV PPPP VVVV ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,830,665ukdxf01,0,57,2,36,26,10,49,49,77,-1,2.790000,3.457438,2.026774,145,0,2,2,0,5,1,89.900000,85,49,1.0,28.0,13.0,že na tomto obrázku vidíme muža ktorý má v ruk...,[fil] že na tomto obrázku vidíme muža ktorý má...,[LONGSIL] [fil] že na tomto obrázku vidíme [LO...,O Eu6 PFis6 SSis6 VKepa+ SSms4 PAms1 VKesc+ E...,že na tento obrázok vidieť muž ktorý mať v ru...,xfilx O Eu6 PFis6 SSis6 VKepa+ SSms4 PAms1 VK...,xlongsilx xfilx O Eu6 PFis6 SSis6 VKepa+ xlon...,OOOO EEEE PPPP SSSS VVVV SSSS PPPP VVVV EEEE ...,xfilx OOOO EEEE PPPP SSSS VVVV SSSS PPPP VVVV...,xlongsilx xfilx OOOO EEEE PPPP SSSS VVVV xlon...
769,831,kixj9dxj01,0,62,3,24,8,16,103,103,173,-1,1.560000,5.201838,2.697249,351,0,3,9,0,2,1,89.940000,182,106,1.0,27.0,17.0,na obrázku vidím muža v kuchyni mi to ta veľmi...,na obrázku vidím muža v kuchyni mi to ta [spk]...,[LONGSIL] na obrázku vidím muža v kuchyni mi t...,Eu6 SSis6 VKesa+ SSms4 Eu6 SSfs6 PPhs3 PFns1 ...,na obrázok vidieť muž v kuchyňa ja|mi to ta v...,Eu6 SSis6 VKesa+ SSms4 Eu6 SSfs6 PPhs3 PFns1 ...,xlongsilx Eu6 SSis6 VKesa+ SSms4 Eu6 SSfs6 PP...,EEEE SSSS VVVV SSSS EEEE SSSS PPPP PPPP PPPP ...,EEEE SSSS VVVV SSSS EEEE SSSS PPPP PPPP PPPP ...,xlongsilx EEEE SSSS VVVV SSSS EEEE SSSS PPPP ...


In [5]:
df

,Unnamed: 0,code,alzheimer,age,education_x,ws_big_big3_pauses,ws_big_big3_pauses_long,ws_big_big3_pauses_short,ws_big_big3_uniquewords,ws_big_big3_uniquewordsnorm,...,big3,big3_2,big3_pauses,big3_tags,big3_lema,big3_2_tags,big3_pauses_tags,big3_first_tag,big3_2_first_tag,big3_pauses_first_tag
0,0,3fiigiv401,0,52,2,25,17,8,81,81,...,a muž v kuchyni varí a má aj pomocníka ten doč...,a muž v kuchyni varí a má aj pomocníka [spk] [...,[LONGSIL] a muž v kuchyni varí a má aj pomocní...,O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T SSms4 PFm...,a|s muž v kuchyňa variť a|s mať aj pomocník t...,O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T SSms4 xsp...,xlongsilx O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T...,OOOO SSSS EEEE SSSS VVVV OOOO VVVV TTTT SSSS ...,OOOO SSSS EEEE SSSS VVVV OOOO VVVV TTTT SSSS ...,xlongsilx OOOO SSSS EEEE SSSS VVVV OOOO VVVV ...
1,1,ysw9a5m701,0,66,2,23,9,14,97,97,...,a ja aj toto je toto je by mohol nevie dal ale...,a ja aj toto je toto je by mohol nevie dal ale...,[LONGSIL] a ja aj toto je toto je by [SHORTSIL...,O PPhs1 T PFns1 VKesc+ PFns1 VKesc+ Y VLesci+...,a|s ja aj toto byť|jesť toto byť|jesť by môcť...,O PPhs1 T PFns1 VKesc+ PFns1 VKesc+ Y VLesci+...,xlongsilx O PPhs1 T PFns1 VKesc+ PFns1 VKesc+...,OOOO PPPP TTTT PPPP VVVV PPPP VVVV YYYY VVVV ...,OOOO PPPP TTTT PPPP VVVV PPPP VVVV YYYY VVVV ...,xlongsilx OOOO PPPP TTTT PPPP VVVV PPPP VVVV ...
2,2,lpvvez2301,2,70,2,17,5,12,63,63,...,niekto tam buchol do tej do toho lustra dieťa ...,niekto tam buchol do tej [fil] do toho lustra ...,[LONGSIL] niekto tam buchol do tej [fil] do to...,PFms1 PD VLdscm+ Eu2 PFfs2 Eu2 PFis2 SSis2 SS...,niekto tam buchnúť do tá do ten luster dieťa ...,PFms1 PD VLdscm+ Eu2 PFfs2 xfilx Eu2 PFis2 SS...,xlongsilx PFms1 PD VLdscm+ Eu2 PFfs2 xfilx Eu...,PPPP PPPP VVVV EEEE PPPP EEEE PPPP SSSS SSSS ...,PPPP PPPP VVVV EEEE PPPP xfilx EEEE PPPP SSSS...,xlongsilx PPPP PPPP VVVV EEEE PPPP xfilx EEEE...
3,3,lpvvweke01,0,71,3,24,17,7,39,39,...,kuchyňa otec so synom chlapec sa snaží pomôcť ...,kuchyňa otec so synom chlapec sa snaží pomôcť ...,[LONGSIL] kuchyňa [LONGSIL] otec so synom [LON...,SSfs1 SSms1 Ev7 SSms7 SSms1 R VKesc+ VId+ SSm...,kuchyňa otec s syn chlapec sa snažiť pomôcť o...,SSfs1 SSms1 Ev7 SSms7 SSms1 R VKesc+ VId+ SSm...,xlongsilx SSfs1 xlongsilx SSms1 Ev7 SSms7 xlo...,SSSS SSSS EEEE SSSS SSSS RRRR VVVV VVVV SSSS ...,SSSS SSSS EEEE SSSS SSSS RRRR VVVV VVVV SSSS ...,xlongsilx SSSS xlongsilx SSSS EEEE SSSS xlong...
4,4,fgl9b8ym01,0,87,2,35,14,21,62,62,...,že no červený manžel no tak najsamprv malý zle...,že no červený manžel [@] no tak najsamprv malý...,[LONGSIL] že [SHORTSIL] no [SHORTSIL] červený ...,O O AAms1x SSms1 O T Dx AAis1x Dx VLdscm+ Eu4...,že no červený manžel no tak najsamprv malý zl...,O O AAms1x SSms1 xzavx O PD Dx AAis1x Dx VLds...,xlongsilx O xshortsilx xshortsilx AAms1x SSms...,OOOO OOOO AAAA SSSS OOOO TTTT DDDD AAAA DDDD ...,OOOO OOOO AAAA SSSS xzavx OOOO PPPP DDDD AAAA...,xlongsilx OOOO xshortsilx xshortsilx AAAA SSS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,827,tbc5xe1q01,0,62,1,23,16,7,51,51,...,no tak dobre chlapček stojí na stoličke naťahu...,no tak dobre chlapček stojí na stoličke naťahu...,[LONGSIL] no tak dobre chlapček stojí na stoli...,O PD Dx SSms1 VKesc+ Eu6 SSfs6 VKesc+ R Eu7 S...,no tak dobre chlapček stáť na stolička naťaho...,O PD Dx SSms1 VKesc+ Eu6 SSfs6 VKesc+ R Eu7 S...,xlongsilx O PD Dx SSms1 VKesc+ Eu6 SSfs6 VKes...,OOOO PPPP DDDD SSSS VVVV EEEE SSSS VVVV RRRR ...,OOOO PPPP DDDD SSSS VVVV EEEE SSSS VVVV RRRR ...,xlongsilx OOOO PPPP DDDD SSSS VVVV EEEE SSSS ...
766,828,lpvve13n01,2,71,3,15,11,4,33,33,...,v kuchyni je och otec asi zo synov ktorých zre...,[fil] v kuchyni [fil] je och [@] otec asi zo s...,[LONGSIL] [fil] v kuchyni [LONGSIL] [fil] je o...,Eu6 SSfs6 VKesc+ J SSms1 T Ev2 SSmp2 PAfp2 T ...,v kuchyňa byť|jesť och otec asi z syn ktorý z...,xfilx Eu6 SSfs6 xfilx VKesc+ J xzavx SSms1 T ...,xlongsilx xfilx Eu6 SSfs6 xlongsilx xfilx VKe...,EEEE SSSS VVVV JJJJ SSSS 

In [6]:
y = df['alzheimer']

# Random Forest

In [7]:
X = df[['moca', 'age', 'education_x', 'ws_big_big3_pauses', 'ws_big_big3_pauses', 'ws_big_big3_uniquewords']]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [10]:
y_pred = clf.predict(X_test)

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Presnosť modelu: {accuracy:.2f}')
print('')
print(classification_report(y_test, y_pred))

Presnosť modelu: 0.94

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       128
           1       0.56      0.50      0.53        10
           2       0.92      0.75      0.83        16

    accuracy                           0.94       154
   macro avg       0.82      0.75      0.78       154
weighted avg       0.94      0.94      0.94       154



# Logistická

In [12]:
X = df[['moca', 'age', 'education_x', 'ws_big_big3_pauses', 'ws_big_big3_pauses', 'ws_big_big3_uniquewords']]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
clf = LogisticRegression(class_weight="balanced", max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [15]:
y_pred = clf.predict(X_test)

In [16]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Presnosť modelu: {accuracy:.2f}')
print('')
print(classification_report(y_test, y_pred))

Presnosť modelu: 0.88

              precision    recall  f1-score   support

           0       1.00      0.89      0.94       134
           1       0.35      0.90      0.50        10
           2       0.89      0.80      0.84        10

    accuracy                           0.88       154
   macro avg       0.75      0.86      0.76       154
weighted avg       0.95      0.88      0.91       154



# K-Means

In [17]:
X = df[['moca', 'age', 'education_x', 'ws_big_big3_pauses', 'ws_big_big3_pauses', 'ws_big_big3_uniquewords']]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
k = len(set(y))
kmeans = KMeans(n_clusters=3, n_init=10)
kmeans.fit(X_train)

KMeans(n_clusters=3, n_init=10)

In [20]:
y_pred = kmeans.predict(X_test)

In [21]:
cluster_map = {}
for i in range(k):
    cluster_labels = y_train[kmeans.labels_ == i]
    if len(cluster_labels) > 0:
        cluster_map[i] = mode(cluster_labels).mode[0]

<ipython-input-21-fb05840046d0>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  cluster_map[i] = mode(cluster_labels).mode[0]


In [22]:
y_pred_mapped = np.array([cluster_map[cluster] for cluster in y_pred])

In [23]:
accuracy = accuracy_score(y_test, y_pred_mapped)
print(f'Presnosť modelu: {accuracy:.2f}')
print(classification_report(y_test, y_pred_mapped))

Presnosť modelu: 0.90
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       139
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00         7

    accuracy                           0.90       154
   macro avg       0.30      0.33      0.32       154
weighted avg       0.81      0.90      0.86       154



C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Lineárna regresia

In [24]:
X = df[['moca', 'age', 'education_x', 'ws_big_big3_pauses', 'ws_big_big3_pauses', 'ws_big_big3_uniquewords']]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression()

In [27]:
y_pred = clf.predict(X_test)

In [28]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [29]:
print(f'Presnosť: {r2:.2f}')
print(f'MAE: {mae:.2f}')
print(f'MSE: {mse:.2f}')

Presnosť: 0.76
MAE: 0.22
MSE: 0.09


# Naivný bayesovský model

In [30]:
X = df[['moca', 'age', 'education_x', 'ws_big_big3_pauses', 'ws_big_big3_pauses', 'ws_big_big3_uniquewords']]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [32]:
clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)
y_pred_nb = clf_nb.predict(X_test)

In [33]:
print(f'Presnosť modelu: {accuracy_score(y_test, y_pred_nb):.2f}')
print(classification_report(y_test, y_pred_nb, zero_division=1))

Presnosť modelu: 0.85
              precision    recall  f1-score   support

           0       0.96      0.90      0.93       144
           1       0.00      0.00      1.00         7
           2       0.20      0.67      0.31         3

    accuracy                           0.85       154
   macro avg       0.39      0.52      0.75       154
weighted avg       0.90      0.85      0.92       154



# Podporný vektorový klasifikátor

In [34]:
X = df[['moca', 'age', 'education_x', 'ws_big_big3_pauses', 'ws_big_big3_pauses', 'ws_big_big3_uniquewords']]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [36]:
clf_svc = SVC(class_weight="balanced", probability=True)
clf_svc.fit(X_train, y_train)
y_pred_svc = clf_svc.predict(X_test)

In [37]:
print(f'Presnosť modelu: {accuracy_score(y_test, y_pred_svc):.2f}')
print(classification_report(y_test, y_pred_svc, zero_division=1))

Presnosť modelu: 0.79
              precision    recall  f1-score   support

           0       0.97      0.80      0.88       136
           1       0.09      0.50      0.16         6
           2       1.00      0.83      0.91        12

    accuracy                           0.79       154
   macro avg       0.69      0.71      0.65       154
weighted avg       0.94      0.79      0.85       154

